In [79]:
from Operations.CO_CompareMinAMI import CO_CompareMinAMI
from Operations.CO_AutoCorr import CO_AutoCorr
from Operations.CO_TranslateShape import CO_TranslateShape
from Operations.CP_wavelet_varch import CP_wavelet_varchg
from Operations.EN_SampEn import EN_SampEn
from Operations.EN_wentropy import EN_wentropy
from PeripheryFunctions.BF_zscore import BF_zscore
import numpy as np
import warnings

In [123]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")
ts4 = np.loadtxt("ts4.txt")

In [14]:
EN_wentropy(ts1, 'sure', 0.1)

np.float64(0.8835824848653626)

In [5]:
CP_wavelet_varchg(ts1)

In [132]:
np.std(7, ddof=1)

np.float64(nan)

In [23]:
np.arange(1+3, 100-3)

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88,
       89, 90, 91, 92, 93, 94, 95, 96])

In [82]:
def CO_TranslateShape(y, shape = 'circle', d = 2, howToMove = 'pts'):
    """
    """
    y = np.array(y, dtype=float)
    N = len(y)

    if y.ndim == 1:
        y = y.reshape(-1, 1)
    elif y.shape[1] > y.shape[0]:
        y = y.T

    # add a time index
    ty = np.column_stack((np.arange(1, N+1), y[:, 0])) # has increasing integers as time in the first column
    #-------------------------------------------------------------------------------
    # Generate the statistics on the number of points inside the shape as it is
    # translated across the time series
    #-------------------------------------------------------------------------------
    if howToMove == 'pts':

        if shape == 'circle':

            r = d # set radius
            w = int(np.floor(r))
            rnge = np.arange(1 + w, N - w + 1)
            NN = len(rnge) # number of admissible points
            np_counts = np.zeros(NN, dtype=int)

            for i in range(NN):
                idx = rnge[i]
                start = idx - w - 1
                end = idx + w
                win = ty[start:end, :]
                difwin = win - ty[idx - 1, :]
                squared_dists = np.sum(difwin**2, axis=1)
                np_counts[i] = np.sum(squared_dists <= r**2)

        elif shape == 'rectangle':

            w = d
            rnge = np.arange(1 + w, N - w + 1)
            NN = len(rnge)
            np_counts = np.zeros(NN, dtype=int)

            for i in range(NN):
                idx = rnge[i]
                start = (idx - w) - 1
                end = (idx + w)
                np_counts[i] = np.sum(
                    np.abs(y[start:end, 0]) <= np.abs(y[i, 0])
                )
        else:
            raise ValueError(f"Unknown shape {shape}. Choose either 'circle' or 'rectangle'")
    else:
        raise ValueError(f"Unknown setting for 'howToMove' input: '{howToMove}'. Only option is currently 'pts'.")

    # compute stats on number of hits inside the shape
    out = {}
    out["max"] = np.max(np_counts)
    out["std"] = np.std(np_counts, ddof=1)
    out["mean"] = np.mean(np_counts)
    
    # count the hits
    vals, hits = np.unique_counts(np_counts)
    max_val = np.argmax(hits)
    out["npatmode"] = hits[max_val]/NN
    out["mode"] = vals[max_val]

    count_types = ["ones", "twos", "threes", "fours", "fives", "sixes", "sevens", "eights", "nines", "tens", "elevens"]
    for i in range(1, 12):
        if 2*w + 1 >= i:
            out[f"{count_types[i-1]}"] = np.mean(np_counts == i)
    

    return out


In [90]:
def SY_SlidingWindow3(y : list, windowStat : str = 'mean', acrossWinStat : str = 'std', numSeg : int = 5, incMove : int = 2) -> dict:

    winLength = np.floor(len(y)/numSeg)
    if winLength == 0:
        warnings.warn(f"Time-series of length {len(y)} is too short for {numSeg} windows")
        return np.nan
    inc = np.floor(winLength/incMove) # increment to move at each step
    # if incrment rounded down to zero, prop it up 
    if inc == 0:
        inc = 1
    
    numSteps = int(np.floor((len(y)-winLength)/inc) + 1)
    qs = np.zeros(numSteps)
    
    # convert a step index (stepInd) to a range of indices corresponding to that window
    def get_window(stepInd: int):
        start_idx = (stepInd) * inc
        end_idx = (stepInd) * inc + winLength
        return np.arange(start_idx, end_idx).astype(int)

    if windowStat == 'mean':
        for i in range(numSteps):
            qs[i] = np.mean(y[get_window(i)])
    elif windowStat == 'std':
        for i in range(numSteps):
            qs[i] = np.std(y[get_window(i)], ddof=1)
    elif windowStat == 'ent':
        warnings.warn(f"{windowStat} not yet implemented")
    elif windowStat == 'apen':
        for i in range(numSteps):
            qs[i] = EN_ApEN(y[get_window(i)], 1, 0.2)
    elif windowStat == 'sampen':
        warnings.warn(f"{windowStat} not yet implemented")
    elif windowStat == 'mom3':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 3)
    elif windowStat == 'mom4':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 4)
    elif windowStat == 'mom5':
        for i in range(numSteps):
            qs[i] = DN_Moments(y[get_window(i)], 5)
    elif windowStat == 'AC1':
        for i in range(numSteps):
            qs[i] = CO_AutoCorr(y[get_window(i)], 1, 'Fourier')
    elif windowStat == 'lillie':
        warnings.warn(f"{windowStat} not yet implemented")
    else:
        raise ValueError(f"Unknown statistic '{windowStat}'")
    

    if acrossWinStat == 'std':
        out = np.std(qs, ddof=1)/np.std(y, ddof=1)
    
    return out
        


In [5]:
SY_SlidingWindow(ts1, 'AC1', 'apen')

np.float64(0.34431508471691297)

In [22]:
def symbols_to_binary(symbols, n):
    n_bits = np.ceil(np.log2(n)) if n > 1 else 1
    binary_sequence = []
    for s in symbols:
        binary_str = format(s, f'0{n_bits}b')
        binary_sequence.extend([int(bit) for bit in binary_str])
    return np.array(binary_sequence)

In [90]:
def lempel_ziv_complexity(data):
    # Convert data to symbols
    s = [int(np.floor(x)) + 1 for x in data]
    bins = max(s) if s else 0  # bins is the maximum symbol value
    n = len(s)
    
    if n == 0:
        return 0.0
    
    c = 1  
    ns = 1  
    nq = 1  
    k = 2  
    
    while k < n:
        is_substring = False
        current_sub = s[ns:ns + nq]
        max_i = ns - nq
        for i in range(0, max_i + 1):
            # Check if the substring starting at i matches current_sub
            if s[i:i + nq] == current_sub:
                is_substring = True
                break
        
        if is_substring:
            nq += 1
        else:
            c += 1
            ns += nq
            nq = 1
        
        k += 1
    
    # Handle potential division by zero if bins is 1
    if bins == 0:
        return 0.0
    try:
        normalized = (c * np.log(n)) / (n * np.log(bins))
    except ZeroDivisionError:
        # If bins is 1, log(1) is 0, leading to division by zero
        normalized = float('inf')
    
    return normalized

In [145]:
def EN_LZcomplexity(x : list, n : int = 2, preProc : str = 'none'):
    if preProc == 'diff':
        x = BF_zscore(np.diff(x))
    else:
        warnings.warn(f"Unknown preprocessing setting {preProc}")
    x = np.array(x).flatten()
    nx = x.size
    # Add small noise to handle ties
    noise = np.finfo(float).eps * np.random.randn(nx)
    x_noisy = x + noise
    xi = np.argsort(x_noisy)
    y = np.arange(1, nx + 1)
    y = np.floor(y * (n / (nx + 1))).astype(int)
    x_sorted_indices = xi
    x[x_sorted_indices] = y

    return lempel_ziv_complexity(x)

In [150]:
EN_LZcomplexity(ts1, 5)

/var/folders/pt/9v0934pn6s1g4klqjbcz65p40000gp/T/ipykernel_34086/2992569531.py:5: UserWarning: Unknown preprocessing setting 
  warnings.warn(f"Unknown preprocessing setting {preProc}")


np.float64(0.12876089022660536)